# 02 – Model Training  
Denne notebook træner en GRU-model til forudsigelse af day-ahead spotpriser baseret på vejr- og tidsdata.  
Der udføres grid search over udvalgte hyperparametre, og den bedste model gemmes.


In [ ]:
# %% 
import pandas as pd
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.preprocessing import StandardScaler
import joblib
import matplotlib.pyplot as plt
import json
import os
import itertools

In [ ]:


# Sikr at mappen findes, ellers opret
os.makedirs("../models", exist_ok=True)

# Indlæs data
df = pd.read_csv("../data/processed/merged.csv", parse_dates=["TimeDK"])


TIME_COL = "TimeDK"
PRICE_COL = "SpotPriceDKK"
FEATURES = ["Temperature", "WindSpeed", "SolarRadiation"]
df = df.sort_values(TIME_COL).reset_index(drop=True)

# Feature engineering
df["hour"] = df[TIME_COL].dt.hour
df["dow"] = df[TIME_COL].dt.dayofweek
FEATURES = FEATURES + ["hour", "dow"]


In [ ]:
# --- Scaling ---
scaler_X = StandardScaler()
scaler_y = StandardScaler()
X = scaler_X.fit_transform(df[FEATURES])
y = scaler_y.fit_transform(df[[PRICE_COL]])




SEQ_LEN = 24
X_seq, y_seq = [], []
for i in range(0, len(df) - SEQ_LEN + 1, SEQ_LEN):
    X_seq.append(X[i:i+SEQ_LEN])        # 24 timers vejrinput
    y_seq.append(y[i:i+SEQ_LEN])        # 24 timers priser for samme vindue
X_seq, y_seq = np.array(X_seq), np.array(y_seq)

fejl_fundet = False

for i, seq in enumerate(X_seq):
    # Inversér kun den enkelte sekvens
    seq_inv = scaler_X.inverse_transform(seq)
    hours = seq_inv[:, 3]  # kolonne 3 = time
    if not np.array_equal(hours, np.arange(24)):
        print(f"Fejl i sekvens {i}: {hours}")
        fejl_fundet = True

if not fejl_fundet:
    print("ingen fejl")



In [ ]:


# --- Split (kronologisk 80/20) ---
split = int(0.8 * len(X_seq))
X_train, X_test = X_seq[:split], X_seq[split:]
y_train, y_test = y_seq[:split], y_seq[split:]




In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train = torch.tensor(X_train, dtype=torch.float32).to(device)
y_train = torch.tensor(y_train, dtype=torch.float32).to(device)
X_test = torch.tensor(X_test, dtype=torch.float32).to(device)
y_test = torch.tensor(y_test, dtype=torch.float32).to(device)

print("Running on:", device)




In [ ]:
# Gem scalers
joblib.dump(scaler_X, "../models/scaler_X.gz")
joblib.dump(scaler_y, "../models/scaler_y.gz")

In [ ]:
# --- Model ---

class GRUModel(nn.Module):
    def __init__(self, input_size, hidden_size=None, num_layers=None, dropout=None):
        super().__init__()
        self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)  # ét output per time

    def forward(self, x):
        out, _ = self.gru(x)
        out = self.fc(out)
        return out



In [ ]:

# --- Træningsfunktion ---
def train_and_validate(model, train_dl, test_dl, optimizer, criterion, model_path, epochs=1000, patience=5):
    best_val = float("inf")
    patience_counter = 0

    for epoch in range(epochs):
        model.train()
        batch_losses = []

        for xb, yb in train_dl:
            optimizer.zero_grad()
            pred = model(xb)
            loss = criterion(pred, yb)
            loss.backward()
            optimizer.step()
            batch_losses.append(loss.item())

        train_mse = sum(batch_losses) / len(batch_losses)

        # --- Validation ---
        model.eval()
        with torch.no_grad():
            val_batch_losses = []
            for xb, yb in test_dl:
                pred = model(xb)
                val_loss = criterion(pred, yb)
                val_batch_losses.append(val_loss.item())
        val_mse = sum(val_batch_losses) / len(val_batch_losses)

        if (epoch+1) % 10 == 0:

          print(f"Epoch {epoch+1}/{epochs}   Val MSE: {val_mse:.5f}     Train MSE: {train_mse:.5f}")

        # --- Early stopping ---
        if val_mse < best_val:
            best_val = val_mse
            patience_counter = 0
            torch.save(model.state_dict(), model_path)
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print(f"Early stopping at epoch {epoch+1}")
                break

    return best_val



In [ ]:
# --- Grid search setup ---
param_grid = {
    "hidden_size": [16, 32, 64],
    "num_layers": [1, 2],
    "dropout": [0.0, 0.2, 0.5],
    "lr": [1e-3, 1e-4],
    "batch_size": [32, 64],
}


criterion = nn.MSELoss()
results = []

best_val = float("inf")
best_model_state = None
best_params = None

for hidden, layers, dropout, lr, batch in itertools.product(
    param_grid["hidden_size"],
    param_grid["num_layers"],
    param_grid["dropout"],
    param_grid["lr"],
    param_grid["batch_size"]):

    print(f"\n=== Træner model: hidden={hidden}, layers={layers}, dropout={dropout}, lr={lr}, batch={batch} ===")

    train_ds = TensorDataset(X_train, y_train)
    test_ds = TensorDataset(X_test, y_test)
    train_dl = DataLoader(train_ds, batch_size=batch, shuffle=False)
    test_dl = DataLoader(test_ds, batch_size=batch, shuffle=False)

    model = GRUModel(X_train.shape[2], hidden, layers, dropout).to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    val_loss = train_and_validate(model, train_dl, test_dl, optimizer, criterion, model_path="tmp.pt")

    results.append({
        "hidden_size": hidden,
        "num_layers": layers,
        "dropout": dropout,
        "lr": lr,
        "batch_size": batch,
        "val_loss": val_loss,
    })

    if val_loss < best_val:
        best_val = val_loss
        best_params = {
            "hidden_size": hidden,
            "num_layers": layers,
            "dropout": dropout,
            "lr": lr,
            "batch_size": batch,
        }
        best_model_state = model.state_dict().copy()

# --- Gem kun bedste model ---
best_path = os.path.join("../models", "best_model.pt")
torch.save(best_model_state, best_path)

# Gem hyperparametre
with open(os.path.join("../models", "best_params.json"), "w") as f:
    json.dump(best_params, f)

print("\nBedste model gemt:")
print(best_params)
print(f"Valideringstab: {best_val:.5f}")


In [ ]:
# Test modellen

with open("../models/best_params.json", "r") as f:
    best_params = json.load(f)

best_model = GRUModel(
    input_size=X_train.shape[2],
    hidden_size=best_params["hidden_size"],
    num_layers=best_params["num_layers"],
    dropout=best_params["dropout"]
).to(device)

best_model.load_state_dict(torch.load("../models/best_model.pt", map_location=device))
best_model.eval()

# --- Forudsigelser ---
best_model.eval()
with torch.no_grad():
    y_pred = best_model(X_test).cpu().numpy()
    y_true = y_test.cpu().numpy()


# --- Rescaler hvis du brugte scaler ---
y_pred_real = scaler_y.inverse_transform(y_pred.reshape(-1, 1)).reshape(y_pred.shape)
y_true_real = scaler_y.inverse_transform(y_true.reshape(-1, 1)).reshape(y_true.shape)

# Plot nogle eksempler
for d in [1,5,50,100]:



  plt.figure(figsize=(8,4))
  plt.plot(y_true_real[d,:,0], label="Faktisk pris")
  plt.plot(y_pred_real[d,:,0], label="Prediktion")
  plt.title(f"Elpris – døgn {d}")
  plt.xlabel("Time på dagen")
  plt.ylabel("Spotpris [DKK/MWh]")
  plt.legend()
  plt.tight_layout()
  plt.show()